In [1]:
import os
%pwd
os.chdir("../")
%pwd

'd:\\Data Science\\END to END Proj\\KidneyDiseaseMLOPS'

In [2]:
from dataclasses import dataclass
from typing import List
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    input_shape: tuple
    epochs: int
    batch_size: int
    validation_split: float
    learning_rate: float



In [3]:
from KidneyClassifier.utils.common import read_yaml, create_directories
from pathlib import Path
from KidneyClassifier.constant import*
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.model_training
        
        create_directories([config.model_dir])

        return ModelTrainingConfig(
            root_dir=Path(config.model_dir),
            trained_model_path=Path(config.trained_model_path),
            input_shape=tuple(params.input_shape),
            epochs=params.epochs,
            batch_size=params.batch_size,
            validation_split=params.validation_split,
            learning_rate=params.learning_rate
        )


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from pathlib import Path

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def build_model(self):
        model = models.Sequential([
            layers.Conv2D(28, (3, 3), activation='relu', 
                         input_shape=self.config.input_shape),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.Flatten(),
            layers.Dense(640, activation='tanh'),
            layers.Dropout(0.5),
            layers.Dense(264, activation='tanh'),
            layers.Dense(64, activation='sigmoid'),
            layers.Dense(4, activation='softmax')
        ])

        optimizer = tf.keras.optimizers.Adam(
            learning_rate=self.config.learning_rate)
        
        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

    def train(self, train_data, val_data):
        if tf.io.gfile.exists(self.config.trained_model_path):
            print(f"Model exists at {self.config.trained_model_path}")
            return tf.keras.models.load_model(self.config.trained_model_path)

        model = self.build_model()
        
        history = model.fit(
            train_data[0], train_data[1],
            validation_data=val_data,
            epochs=self.config.epochs,
            batch_size=self.config.batch_size
        )

        # Save as SavedModel format
        model.save(self.config.trained_model_path) 
        print(f"Saved model to {self.config.trained_model_path}")
        
        return history

In [5]:
import numpy as np
from pathlib import Path

try:
    # Initialize configuration
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    
    # Load preprocessed data
    processed_data_path = Path("artifacts/data_preprocessing/preprocessed_data.npz")
    processed_data = np.load(processed_data_path)
    X_train, X_test = processed_data['X_train'], processed_data['X_test']
    y_train, y_test = processed_data['y_train'], processed_data['y_test']
    
    # Initialize and run model training
    model_trainer = ModelTrainer(config=model_training_config)  # Fixed: Using ModelTrainer instead of ModelTrainingConfig
    history = model_trainer.train((X_train, y_train), (X_test, y_test))  # Fixed: Using train() method
    
    print("Training completed successfully!")
    print(f"Model saved to: {model_training_config.trained_model_path}")
    
except Exception as e:
    print(f"Error during model training: {str(e)}")
    raise e

[2025-07-25 01:00:02,632: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-25 01:00:02,641: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-25 01:00:02,644: INFO: common: created directory at: artifacts]
[2025-07-25 01:00:02,644: INFO: common: created directory at: artifacts/model_training]
Epoch 1/50
508/508 [==============================] - 33s 58ms/step - loss: 0.4205 - accuracy: 0.8329 - val_loss: 0.0455 - val_accuracy: 0.9906
Epoch 2/50
508/508 [==============================] - 30s 58ms/step - loss: 0.0529 - accuracy: 0.9857 - val_loss: 0.0658 - val_accuracy: 0.9756
Epoch 3/50
508/508 [==============================] - 32s 62ms/step - loss: 0.0302 - accuracy: 0.9914 - val_loss: 0.0526 - val_accuracy: 0.9830
Epoch 4/50
508/508 [==============================] - 32s 62ms/step - loss: 0.0206 - accuracy: 0.9947 - val_loss: 0.0088 - val_accuracy: 0.9978
Epoch 5/50
508/508 [==============================] - 33s 64ms/step - loss: 0.0247 - acc